# 📊 Guía Práctica 1 - Análisis de Compras Públicas (Ecuador)
**Autor:** Wendy Llivichuzhca

Esta notebook documenta y ejecuta las visualizaciones de datos de compras públicas del Ecuador.
Aquí aprenderás a:
- Cargar datos desde la API de Compras Públicas.
- Preprocesar y limpiar los datos.
- Visualizar estadísticas descriptivas.
- Crear gráficos interactivos con Plotly.
- Exportar los datos filtrados.

Esta notebook está diseñada como un **tutorial paso a paso**, para que puedas modificar y experimentar con los filtros y análisis.

In [ ]:
# -----------------------------
# 0️⃣ Librerías necesarias
# -----------------------------
import pandas as pd
import requests
import plotly.express as px
from datetime import datetime
from IPython.display import display
from ipywidgets import interact, widgets

## 1️⃣ Configuración de filtros
Aquí definimos los valores posibles para los filtros: año, provincia, tipo de contratación y palabra clave. Puedes modificarlos directamente o usar los widgets interactivos más abajo.

In [ ]:
# Valores posibles para filtros
años = [str(a) for a in range(2015, 2026)]
provincias = ["Todos", "AZUAY", "BOLÍVAR", "CAÑAR", "CARCHI", "CHIMBORAZO", "COTOPAXI",
              "EL ORO", "ESMERALDAS", "GALÁPAGOS", "GUAYAS", "IMBABURA", "LOJA", "LOS RÍOS",
              "MANABÍ", "MORONA SANTIAGO", "NAPO", "ORELLANA", "PASTAZA", "PICHINCHA",
              "SANTA ELENA", "SANTO DOMINGO DE LOS TSÁCHILAS", "SUCUMBÍOS", "TUNGURAHUA",
              "ZAMORA CHINCHIPE"]
tipos_contratacion = ["Todos", "Subasta Inversa Electrónica", "Menor Cuantía", "Cotización",
                      "Contratación directa", "Licitación", "Catálogo electrónico", "Bienes y Servicios únicos"]

# Valores por defecto
anio = '2022'
provincia = 'Todos'
tipo = 'Todos'
palabra = ''

### 🎛️ Filtros interactivos (opcional)
Si deseas seleccionar filtros sin editar el código, puedes usar los widgets interactivos.

In [ ]:
def seleccionar_filtros():
    a = widgets.Dropdown(options=años, value='2022', description='Año:')
    p = widgets.Dropdown(options=provincias, value='Todos', description='Provincia:')
    t = widgets.Dropdown(options=tipos_contratacion, value='Todos', description='Tipo:')
    k = widgets.Text(value='', description='Palabra:')
    return a, p, t, k

anio_widget, provincia_widget, tipo_widget, palabra_widget = seleccionar_filtros()
display(anio_widget, provincia_widget, tipo_widget, palabra_widget)

## 2️⃣ Función para cargar datos desde la API
Esta función hace la solicitud a la API de Compras Públicas usando los filtros seleccionados y devuelve un DataFrame limpio.

In [ ]:
def cargar_datos(year, region, tipo, palabra):
    """Carga datos de la API de Compras Públicas con filtros"""
    url = "https://datosabiertos.compraspublicas.gob.ec/PLATAFORMA/api/get_analysis"
    params = {
        "year": year,
        "region": None if region == "Todos" else region,
        "type": None if tipo == "Todos" else tipo,
        "keyword": palabra if palabra else None
    }
    resp = requests.get(url, params=params)
    if resp.status_code != 200:
        print("❌ Error al consultar la API")
        return pd.DataFrame()
    data = resp.json()
    if not data:
        print("⚠️ No se encontraron datos con estos filtros")
        return pd.DataFrame()
    df = pd.DataFrame(data)
    return df

## 3️⃣ Carga y preprocesamiento de datos
Se realiza limpieza, renombrado de columnas, conversión de tipos y creación de nuevas columnas para análisis por mes y año.

In [ ]:
# Usar valores de widgets si se desea
anio = anio_widget.value
provincia = provincia_widget.value
tipo = tipo_widget.value
palabra = palabra_widget.value

df = cargar_datos(anio, provincia, tipo, palabra)

if df.empty:
    print("⚠️ No hay datos para los filtros seleccionados")
else:
    rename_map = {"province": "region", "type": "internal_type", "amount": "total", "date": "date"}
    df.rename(columns={k:v for k,v in rename_map.items() if k in df.columns}, inplace=True)

    if 'total' in df.columns:
        df['total'] = pd.to_numeric(df['total'], errors='coerce')
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        df['month'] = df['date'].dt.month
        df['year'] = df['date'].dt.year

    df.drop_duplicates(inplace=True)
    df.dropna(subset=['total'], inplace=True)

    display(df.head())

## 4️⃣ Métricas generales
Calculamos total de registros, monto total y promedio, y mostramos un resumen estadístico.

In [ ]:
if not df.empty:
    total_registros = len(df)
    monto_total = df['total'].sum()
    monto_promedio = df['total'].mean()

    print(f"Total de registros: {total_registros}")
    print(f"Monto total: ${monto_total:,.2f}")
    print(f"Monto promedio: ${monto_promedio:,.2f}")

    display(df.describe())

## 5️⃣ Visualizaciones Interactivas con Plotly
Se crean gráficos de barras, línea y pastel para analizar montos totales, evolución mensual y proporción de contratos.

In [ ]:
# a) Monto total por tipo de contratación
if {'internal_type','total'}.issubset(df.columns):
    df_tipo = df.groupby('internal_type')['total'].sum().reset_index()
    fig1 = px.bar(df_tipo, x='internal_type', y='total',
                  title='Monto total por tipo de contratación',
                  color_discrete_sequence=['#36BF8D'])
    fig1.show()

In [ ]:
# b) Evolución mensual de montos
if {'month','total'}.issubset(df.columns):
    df_mes = df.groupby('month')['total'].sum().reset_index()
    fig2 = px.line(df_mes, x='month', y='total',
                   title='Evolución mensual de montos totales',
                   markers=True, color_discrete_sequence=['#36BF8D'])
    fig2.show()

In [ ]:
# c) Barras apiladas tipo × mes
if {'month','internal_type','total'}.issubset(df.columns):
    df_tipo_mes = df.groupby(['month','internal_type'])['total'].sum().reset_index()
    fig3 = px.bar(df_tipo_mes, x='month', y='total', color='internal_type', barmode='stack',
                  title='Monto total por tipo de contratación y mes')
    fig3.show()

In [ ]:
# d) Proporción de contratos
if 'internal_type' in df.columns:
    fig4 = px.pie(df, names='internal_type',
                  title='Proporción de contratos por tipo de contratación',
                  color_discrete_sequence=px.colors.sequential.Greens)
    fig4.show()

In [ ]:
# e) Montos totales por tipo de contratación por año
if {'year','internal_type','total'}.issubset(df.columns):
    df_year = df.groupby(['year','internal_type'])['total'].sum().reset_index()
    fig5 = px.bar(df_year, x='year', y='total', color='internal_type', barmode='stack',
                  title='Montos totales por tipo de contratación por año')
    fig5.show()

## 6️⃣ Exportación de datos
Puedes exportar los datos filtrados a CSV para análisis externo.

In [ ]:
if not df.empty:
    filename = f"compras_publicas_{anio}.csv"
    df.to_csv(filename, index=False, encoding='utf-8')
    print(f"✅ Datos exportados a {filename}")

## 7️⃣ Conclusiones
- Los tipos de contratación más utilizados se destacan en los gráficos.
- Se observan variaciones mensuales de los montos.
- La relación entre monto y número de contratos evidencia concentración de gasto.
- La descarga de datos permite análisis más profundo fuera de la notebook.
- Esta notebook facilita la comprensión visual y analítica de la gestión de compras públicas en Ecuador.